In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tensorflow.keras import layers
from tensorflow import keras


tf.keras.backend.clear_session()  # For easy reset of notebook state.
print(cv2.__version__)
print(tf.__version__)
assert tf.executing_eagerly() == True

class_names = ["black", "blue", "brown", "gold", "green", "orange", "purple", "red", "silver", "white"]
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHN = 3


def generate_generator(generator, path, batch_size = 16, img_height = IMG_HEIGHT, img_width = IMG_WIDTH):

        gen = generator.flow_from_directory(path,
                                            classes = class_names,
                                            target_size = (img_height,img_width),
                                            batch_size = batch_size,
                                            shuffle=True, 
                                            seed=7)
        while True:
            X,y = gen.next()  
            data = []
            for img in X:
                chans = cv2.split(img)
                colors = ("r", "b", "g")
                features = []
                # loop over the image channels
                for (chan, color) in zip(chans, colors):
                    hist = cv2.calcHist([chan], [0], None, [256], [1, 256])
                    features.extend(hist)
                features = np.squeeze(np.array(features))
                data.append(features)
            data = np.array(data)/100.0
            print(data.shape)
            yield data, y #Yield both images and their mutual label
                
class Dataloader:
    def __init__(self, data_path,  batch_size = 16):
        
       	train_imgen = keras.preprocessing.image.ImageDataGenerator(rotation_range = 10,\
										width_shift_range = 0.35, height_shift_range = 0.35,\
										horizontal_flip = True)

        test_imgen = keras.preprocessing.image.ImageDataGenerator()

        self.train_generator = generate_generator(train_imgen,
                                               path = str(data_path) + "train/",
                                               batch_size=batch_size)       

        self.val_generator = generate_generator(test_imgen,
                                              path = str(data_path)+ "val/",
                                              batch_size=batch_size)              

        
    def load_image(self, val = False):
        if val:
            return next(self.val_generator)
        else:
            return next(self.train_generator)
    
    def load_dl(self):
        return [self.train_generator, self.val_generator]
    
    
def imshow(inp, title=None):
    """Imshow for Tensor."""

    #inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    #inp = std * inp + mean
    #inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
dl = Dataloader("./mixed_vehicle/")
inputs, labels = dl.load_image()
classes = [np.where(r==1)[0][0] for r in labels]

lr = 0.01

def CarColorClassificationNet(): 
    
    model = tf.keras.Sequential()
    model.add(layers.Input(shape = (256*3), dtype = 'float32', name = "input" ))
    model.add(layers.Dense(32, activation= 'relu', name = "fc1", kernel_initializer = 'glorot_uniform'))
    model.add(layers.Dense(10, activation= 'softmax', name = "output", kernel_initializer = 'glorot_uniform'))
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
    
    return model

#create DNN model


batch_size = 16
trainset_size = 3087
valset_size = 575
epochs = 100
dl = Dataloader("./mixed_vehicle/")
train_generator, val_generator = dl.load_dl()
checkpoint_path = "training_hist_new/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


model = CarColorClassificationNet()
keras.utils.plot_model(model, 'vehicle_color_classification.png')




4.1.2
1.14.0
Found 3129 images belonging to 10 classes.
(16, 768)
Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [16]:
model.fit_generator(train_generator,
                                steps_per_epoch=trainset_size/batch_size,
                                epochs = epochs,
                                validation_data = val_generator,
                                validation_steps = valset_size/batch_size,
                                use_multiprocessing = True,
                                shuffle=True,
                                callbacks=[cp_callback])

Found 3087 images belonging to 10 classes.
Epoch 1/100
192/192 [============================>.] - ETA: 0s - loss: 1.7505 - categorical_accuracy: 0.4535Found 575 images belonging to 10 classes.

Epoch 00001: saving model to training_hist_new/cp.ckpt
193/192 [==============================] - 58s 301ms/step - loss: 1.7476 - categorical_accuracy: 0.4538 - val_loss: 1.5613 - val_categorical_accuracy: 0.4974
Epoch 2/100
192/192 [============================>.] - ETA: 0s - loss: 1.2740 - categorical_accuracy: 0.5479Found 575 images belonging to 10 classes.

Epoch 00002: saving model to training_hist_new/cp.ckpt
193/192 [==============================] - 55s 287ms/step - loss: 1.2747 - categorical_accuracy: 0.5478 - val_loss: 1.4584 - val_categorical_accuracy: 0.5287
Epoch 3/100
192/192 [============================>.] - ETA: 0s - loss: 1.1827 - categorical_accuracy: 0.5934Found 575 images belonging to 10 classes.

Epoch 00003: saving model to training_hist_new/cp.ckpt
193/192 [==============

Process Keras_worker_ForkPoolWorker-37:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)


KeyboardInterrupt: 

  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/charles/.local/lib/python3.6/site-packages/tensorflow/python/keras/utils/data_utils.py", line 828, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "<ipython-input-15-662393544ba2>", line 33, in generate_generator
    X,y = gen.next()
  File "/home/charles/.local/lib/python3.6/site-packages/keras_preprocessing/image/iterator.py", line 116, in next
    return self._get_batches_of_transformed_samples(index_array)
  File "/home/charles/.local/lib/python3.6/site-packages/keras_preprocessing/image/iterator.py", line 238, in _get_batches_of_transformed_samples
    x = self.image_data_generator.apply_transform(x, params)
  File "/home/charles/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py", line 870, in apply_transform
    order=self.interpolation_order)
  File "/home/charles/.local/lib/python3.6/site-packages/ke